In [23]:
from pyspark.sql import SparkSession
from pyhive import hive

In [10]:
from pyspark.sql.functions import from_unixtime, col

In [24]:
# Kết nối tới Hive
conn = hive.Connection(host='localhost', port=10000)
cursor = conn.cursor()
    
    
# Tạo database nếu chưa có
query = "CREATE DATABASE IF NOT EXISTS healthcare_System"

cursor.execute(query)

cursor.close()
conn.close()

In [26]:
spark = SparkSession.builder \
    .appName("MySQLtoHDFS") \
    .config("spark.jars", "/home/hduser/Downloads/test/usr/share/java/mysql-connector-j-8.4.0.jar") \
    .getOrCreate()
    
# Kết nối tới Hive
conn = hive.Connection(host='localhost', port=10000)
cursor = conn.cursor()

for collection in ['claims', 'disease', 'group_subgroup', 'groups_tble', 'hospital_details', 'patient_details', 'subgroup', 'subscriber']:
    try:
        # Đọc dữ liệu từ MySQL
        df = spark.read \
            .format("jdbc") \
            .option("url", "jdbc:mysql://localhost:3306/healthcare") \
            .option("driver", "com.mysql.cj.jdbc.Driver") \
            .option("dbtable", collection) \
            .option("user", "root") \
            .option("password", "Mta@2024") \
            .load()
            
        # Show dữ liệu
        df.show()
            
        listDateNeedToCovert = [['claims', ['claim_date']], ['patient_details', ['patient_birth_date']], ['subscriber', ['birth_date', 'eff_date', 'term_date']]]
        
        # # Covert từ DateWritableV2 sang TIMESTAMP
        # for collectionToCovert, columnToConvert in listDateNeedToCovert:
        #     if collection == collectionToCovert:
        #         for column in columnToConvert:
        #             df = df.withColumn(column, from_unixtime(col(column).cast("bigint") / 1000))
            
        # Ghi dữ liệu vào HDFS
        df.write.parquet(f"hdfs://master:9000/user/hduser/{collection}")
        
        # Lấy tên các cột và kiểu dữ liệu
        columns = [(col.name, col.dataType) for col in df.schema]

        # Tạo định nghĩa cột cho bảng Hive
        column_definitions = ", ".join([f"{col} {str(dtype).upper().replace('DOUBLETYPE', 'DOUBLE').replace('STRINGTYPE', 'STRING').replace('INTEGERTYPE', 'INT').replace('DATETYPE', 'DATE')}" for col, dtype in columns])

        # Tạo câu lệnh CREATE EXTERNAL TABLE
        create_table_query = f"""
        CREATE EXTERNAL TABLE IF NOT EXISTS healthcare_System.{collection} (
            {column_definitions}
        )
        STORED AS PARQUET
        LOCATION 'hdfs://master:9000/user/hduser/{collection}'
        """

        try:
            # Thực thi câu lệnh CREATE EXTERNAL TABLE
            cursor.execute(create_table_query)
            print("Tạo bảng Hive thành công!")
        except Exception as e:
            print(f"Lỗi khi tạo bảng Hive: {e}")
            
    except Exception as e:
        print(e)

spark.stop()
cursor.close()
conn.close()

+--------+----------+----------------+----------+-----------------+----------------+------------+----------+
|claim_id|patient_id|    disease_name|    sub_id|claim_or_rejected|      claim_type|claim_amount|claim_date|
+--------+----------+----------------+----------+-----------------+----------------+------------+----------+
|       1|    187158|    Galactosemia|SUBID10000|                N| claims of value|     79874.0|1949-03-14|
|       2|    112766|  Bladder cancer|SUBID10001|                N|claims of policy|    151142.0|1970-03-16|
|       3|    199252|   Kidney cancer|SUBID10002|                N| claims of value|     59924.0|2008-02-03|
|       4|    133424|         Suicide|SUBID10003|                N|  claims of fact|    143120.0|1995-02-08|
|       5|    172579|    Food allergy|SUBID10004|                Y| claims of value|    168634.0|1967-05-23|
|       6|    171320|        Whiplash|SUBID10005|                N|claims of policy|     64840.0|1991-10-04|
|       7|    10779

Tạo bảng Hive thành công!
+----------+--------------------+---------+
|disease_id|        disease_name|subgrp_id|
+----------+--------------------+---------+
|    110001|          Beriberi\n|     S101|
|    110002|            Scurvy\n|     S101|
|    110003|            Goitre\n|     S101|
|    110004|      Osteoporosis\n|     S101|
|    110005|           Rickets\n|     S101|
|    110006|           Anaemia\n|     S101|
|    110007|         Fractures\n|     S102|
|    110008|      Heart Attack\n|     S102|
|    110009|             Burns\n|     S102|
|    110010|           Choking\n|     S102|
|    110011|            Stroke\n|     S102|
|    110012|    Food Poisoning\n|     S102|
|    110013|            Asthma\n|     S103|
|    110014|          Glaucoma\n|     S103|
|    110015|          Diabetes\n|     S103|
|    110016|           Amnesia\n|     S103|
|    110017|       Parasomnias\n|     S103|
|    110018|Neurocognitive di...|     S103|
|    110019|           Vertigo\n|     S104|
|    1

In [27]:
spark = SparkSession.builder \
    .appName("MySQLtoHDFS") \
    .config("spark.jars", "/home/hduser/Downloads/test/usr/share/java/mysql-connector-j-8.4.0.jar") \
    .getOrCreate()

In [28]:
# TEST
# Đọc dữ liệu parquet từ HDFS
spark.read.parquet("hdfs://master:9000/user/hduser/claims").show()

spark.stop()

+--------+----------+----------------+----------+-----------------+----------------+------------+----------+
|claim_id|patient_id|    disease_name|    sub_id|claim_or_rejected|      claim_type|claim_amount|claim_date|
+--------+----------+----------------+----------+-----------------+----------------+------------+----------+
|       1|    187158|    Galactosemia|SUBID10000|                N| claims of value|     79874.0|1949-03-14|
|       2|    112766|  Bladder cancer|SUBID10001|                N|claims of policy|    151142.0|1970-03-16|
|       3|    199252|   Kidney cancer|SUBID10002|                N| claims of value|     59924.0|2008-02-03|
|       4|    133424|         Suicide|SUBID10003|                N|  claims of fact|    143120.0|1995-02-08|
|       5|    172579|    Food allergy|SUBID10004|                Y| claims of value|    168634.0|1967-05-23|
|       6|    171320|        Whiplash|SUBID10005|                N|claims of policy|     64840.0|1991-10-04|
|       7|    10779

In [29]:
# Kết nối tới Hive
conn = hive.Connection(host='localhost', port=10000)
cursor = conn.cursor()

query = f"show databases"

try:
    cursor.execute(query)
    result = cursor.fetchall()
    print(result)
except Exception as e:
    print(f"Lỗi khi đọc dữ liệu từ bảng Hive: {e}")
        
# Đóng kết nối
cursor.close()
conn.close()

[('adidasshop',), ('default',), ('eshop',), ('healthcare_system',)]


In [30]:
# Kết nối tới Hive
conn = hive.Connection(host='localhost', port=10000)
cursor = conn.cursor()

for collection in ['claims', 'disease', 'group_subgroup', 'groups_tble', 'hospital_details', 'patient_details', 'subgroup', 'subscriber']:
    # Thử đọc dữ liệu từ bảng groups_tble bằng pyhive
    print(f"Đọc dữ liệu từ bảng {collection}")
    query = f"SELECT * FROM healthcare_System.{collection}"

    try:
        cursor.execute(query)
        result = cursor.fetchall()
        print(result)
    except Exception as e:
        print(f"Lỗi khi đọc dữ liệu từ bảng Hive: {e}")
        
# Đóng kết nối
cursor.close()
conn.close()

Đọc dữ liệu từ bảng claims
[(1, 187158, 'Galactosemia', 'SUBID10000', 'N', 'claims of value', 79874.0, '1949-03-14'), (2, 112766, 'Bladder cancer', 'SUBID10001', 'N', 'claims of policy', 151142.0, '1970-03-16'), (3, 199252, 'Kidney cancer', 'SUBID10002', 'N', 'claims of value', 59924.0, '2008-02-03'), (4, 133424, 'Suicide', 'SUBID10003', 'N', 'claims of fact', 143120.0, '1995-02-08'), (5, 172579, 'Food allergy', 'SUBID10004', 'Y', 'claims of value', 168634.0, '1967-05-23'), (6, 171320, 'Whiplash', 'SUBID10005', 'N', 'claims of policy', 64840.0, '1991-10-04'), (7, 107794, 'Sunbathing', 'SUBID1006', 'N', 'claims of fact', 26800.0, '1991-03-26'), (8, 130339, 'Drug consumption', 'SUBID10007', 'N', 'claims of value', 177186.0, '1946-09-05'), (9, 110377, 'Dengue', 'SUBID10008', 'N', 'claims of fact', 141123.0, '1966-06-20'), (10, 149367, 'Head banging', 'SUBID10009', 'N', 'claims of value', 88540.0, '1945-12-29'), (11, 156168, 'Fanconi anaemia', 'SUBID1010', 'N', 'claims of value', 29150.0, 

# Đọc dữ liệu từ HDFS

In [1]:
# # Import the necessary libraries
# from pyspark.sql import SparkSession

# # Create a SparkSession
# spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/02 12:01:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# # Execute a query to select data from the table
# df = spark.read.parquet("hdfs://master:9000/user/hduser/groups_tble")

# df.createOrReplaceTempView("groups")

In [9]:
# spark.sql("SELECT * FROM groups").show()

+------+------+--------------------+---------------+---------+--------+-------+--------+
|grp_sk|grp_id|            grp_name|premium_written|     city|zip_code|country|grp_type|
+------+------+--------------------+---------------+---------+--------+-------+--------+
|     1|GRP101|Life Insurance Co...|          72000|   Mumbai|  482018|  India|   Govt.|
|     2|GRP102|HDFC Standard Lif...|          45000|   Mumbai|  482049|  India| Private|
|     3|GRP103|Max Life Insuranc...|          64000|    Delhi|  482030|  India| Private|
|     4|GRP104|ICICI Prudential ...|          59000|   Mumbai|  482028|  India| Private|
|     5|GRP105|Kotak Mahindra Li...|          37000|   Mumbai|  482014|  India| Private|
|     6|GRP106|Aditya Birla Sun ...|          89000|   Mumbai|  482011|  India| Private|
|     7|GRP107|TATA AIG Life Ins...|          70000|   Mumbai|  482006|  India| Private|
|     8|GRP108|SBI Life Insuranc...|          52000|   Mumbai|  482034|  India| Private|
|     9|GRP109|Exide 

In [10]:
# spark.stop()